# Model Selection

### API Key를 불러옵니다.

이전과 파일에서처럼 `helper.py` 파일의 api key를 꼭 확인해야 합니다.

In [ ]:
from helper import load_mistral_api_key
api_key, dlai_endpoint = load_mistral_api_key(ret_key=True)

미스트랄 모델 관련 함수를 직접 정의합니다.

In [ ]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

def mistral(user_message, model="mistral-small-latest", is_json=False):
    client = MistralClient(api_key=api_key, endpoint=dlai_endpoint)
    messages = [ChatMessage(role="user", content=user_message)]

    if is_json:
        chat_response = client.chat(
            model=model, messages=messages, response_format={"type": "json_object"}
        )
    else:
        chat_response = client.chat(model=model, messages=messages)

    return chat_response.choices[0].message.content

본 챕터에서는 미스트랄에서 제공하는 여러 모델들을 적재적소에 활용하는 내용을 다루고 있습니다.

각 모델을 사용하는 비용(시간, 금액, 컴퓨팅 파워 등)을 고려했을 때, 최소한의 비용으로 동일한 결과를 얻기 위함이죠.

즉, small 사이즈의 모델로도 충분한 작업을 굳이 large 모델로 처리할 이유는 없다는 뜻입니다.

## Mistral Small

간단한 태스크, 빠른 추론, 적은 비용을 위한 작업에 적합합니다.
- 분류

In [ ]:
prompt = """
Classify the following email to determine if it is spam or not.
Only respond with the exact text "Spam" or "Not Spam". 

# Email:
🎉 Urgent! You've Won a $1,000,000 Cash Prize! 
💰 To claim your prize, please click on the link below: 
https://bit.ly/claim-your-prize
"""

In [ ]:
mistral(prompt, model="mistral-small-latest")

## Mistral Medium

언어 변환과 같은 중간 수준의 태스크에 적합합니다.
- 주어진 맥락을 바탕으로 텍스트를 구성합니다. (예를 들어 구매 정보를 바탕으로 고객 서비스 이메일을 작성할 수 있습니다.)

In [ ]:
prompt = """
Compose a welcome email for new customers who have just made 
their first purchase with your product. 
Start by expressing your gratitude for their business, 
and then convey your excitement for having them as a customer. 
Include relevant details about their recent order. 
Sign the email with "The Fun Shop Team".

Order details:
- Customer name: Anna
- Product: hat 
- Estimate date of delivery: Feb. 25, 2024
- Return policy: 30 days
"""

In [ ]:
response_medium = mistral(prompt, model="mistral-medium-latest")

In [ ]:
print(response_medium)

## Mistral Large: 

고난이도의 추론 능력을 요하는 복잡한 태스크에 적합합니다.
- 숫자를 이용한 수학적 추론

In [ ]:
prompt = """
Calculate the difference in payment dates between the two \
customers whose payment amounts are closest to each other \
in the following dataset. Do not write code.

# dataset: 
'{
  "transaction_id":{"0":"T1001","1":"T1002","2":"T1003","3":"T1004","4":"T1005"},
    "customer_id":{"0":"C001","1":"C002","2":"C003","3":"C002","4":"C001"},
    "payment_amount":{"0":125.5,"1":89.99,"2":120.0,"3":54.3,"4":210.2},
"payment_date":{"0":"2021-10-05","1":"2021-10-06","2":"2021-10-07","3":"2021-10-05","4":"2021-10-08"},
    "payment_status":{"0":"Paid","1":"Unpaid","2":"Paid","3":"Paid","4":"Pending"}
}'
"""

작은 모델로 추론했을 때의 결과와 비교해 봅니다.

In [ ]:
response_small = mistral(prompt, model="mistral-small-latest")

In [ ]:
print(response_small)

In [ ]:
response_large = mistral(prompt, model="mistral-large-latest")

In [ ]:
print(response_large)

## Expense reporting task (비용 보고)

In [ ]:
transactions = """
McDonald's: 8.40
Safeway: 10.30
Carrefour: 15.00
Toys R Us: 20.50
Panda Express: 10.20
Beanie Baby Outlet: 25.60
World Food Wraps: 22.70
Stuffed Animals Shop: 45.10
Sanrio Store: 85.70
"""

prompt = f"""
Given the purchase details, how much did I spend on each category:
1) restaurants
2) groceries
3) stuffed animals and props
{transactions}
"""

In [ ]:
response_small = mistral(prompt, model="mistral-small-latest")
print(response_small)

In [ ]:
response_large = mistral(prompt, model="mistral-large-latest")
print(response_large)

## Writing and checking code

large 모델은 코드 작성 능력도 뛰어납니다.

In [ ]:
user_message = """
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

You can return the answer in any order.

Your code should pass these tests:

assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]
"""

In [ ]:
print(mistral(user_message, model="mistral-large-latest"))

### 모델이 작성해준 코드를 한 번 실행해 보세요

Here is the code that was output at the time of filming:
```Python
def twoSum(nums, target):
    seen = {}
    for i, num in enumerate(nums):
        complement = target - num
        if complement in seen:
            return [seen[complement], i]
        seen[num] = i
```
- Also try running the assert statements in the original prompt
```Python
assert twoSum([2,7,11,15], 9) == [0,1]
assert twoSum([3,2,4], 6) == [1,2]
assert twoSum([3,3], 6) == [0,1]
```

## 영어, 프랑스어, 스페인어, 독일어, 이탈리아어에 능숙합니다.
- 이는 미스트랄 모델을 번역 작업에 쉽게 이용할 수 있다는 걸 의미합니다.
- 예를 들어 만약 당신이 native 스페인어 화자라면 당신은 미스트랄 모델과 스페인어로 쉽게 소통 가능합니다.

In [ ]:
user_message = """
Lequel est le plus lourd une livre de fer ou un kilogramme de plume
"""

In [ ]:
print(mistral(user_message, model="mistral-large-latest"))

## 호출 가능한 미스트랄 모델 목록

API call을 통해 사용 가능한 두 개의 오픈 소스 모델이 있습니다.
또한 아래 모델 리스트 역시 사용 가능합니다.
```
open-mistral-7b
open-mixtral-8x7b
mistral-small-latest
mistral-medium-latest
mistral-large-latest
```

예시:
```Python
mistral(prompt, model="open-mixtral-8x7b")
```